# Member Engagement Targeting: Logistic Regression

## Step 1: Import necessary Python modules and set file paths

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, RepeatedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier

import re
import seaborn as sns
from itertools import product

fp_raw_data  = '/nfs/home/btl1613/team/practicum/2020-lhic/raw/wmb_trigger_data.csv'
fp_pre_covid = '/nfs/home/btl1613/team/practicum/2020-lhic/sandbox_cleaned/cleaned_pre_covid.csv'
fp_in_covid  = '/nfs/home/btl1613/team/practicum/2020-lhic/sandbox_cleaned/cleaned_in_covid.csv'
fp_all_data  = '/nfs/home/btl1613/team/practicum/2020-lhic/sandbox_cleaned/cleaned_all.csv'
fp_output    = 'member_targets_output.csv'

/opt/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Step 2: Data Preprocessing

We began by loading the raw data and performing the following cleaning steps:
- Creating pre- and post-COVID-19 segments (based on the WHO's announcement on March 11, 2020)
- Dropping fields that weren't necessary, overly complex, or redundant for modeling per our discussions with the DS team at LHIC
- Performing one-hot encoding for categorical or 'flag' variables
- Standardizing continuous variables so as not to introduce inadvertent skew in variable weights and importance
- Deduplicating rows
- Sorting the dataset into different time sets relative to COVID-19

In [2]:
df1 = pd.read_csv(fp_raw_data)

# Create pre- and post-COVID flags based on WHO declaration of COVID-19 pandemic on March 11, 2020
df1['screen_dt_deid']= pd.to_datetime(df1['screen_dt_deid'])
df1['pre_COVID'] = np.where(df1['screen_dt_deid']< '2020-03-11', 1, 0)

# Drop unhelpful columns:

             # We can scrap these because we'll suffer from dimensionality issues and
             # we don't need to alter the Q-scores with the dates
drop_cols = ['Unnamed: 0','A1270_PersonicxClassicRefresh',\
             'condition_cd','primy_diag_flag','bh_diag_cat',
             'bh_pgm','run_end_deid','strt_dt_deid','inc_end_deid',
             # We also don't know these variables before they engage, so also not helpful.
             'screen_dt_deid', 'close_dt_deid', 'case_closure_rsn', 
             'INCURD_DT_deid', 'case_id_deid', 'mid_deid', 'Follow_Up_Task_Created', 
             'Successfully_Resolved', 'Not_Valued', 'Unable_to_resolve', 'utr', 
             'refused', 'withdrew',
             # RULE_ID is redundant, so dropping
             'RULE_ID',
             # These variables shouldn't have a meaningful relationship to engagement
             'oc_CCS_LVL_1', 'icd_vrsn_cd'
             ]
df2 = df1.drop(columns=drop_cols)

# One-hot encoding:
looping_list =['Ovarian_Cancer_deid', 'CVA_and_Cerebral_Aneurysm_deid',
               'Hepatitis_C_deid', 'Huntingtons_Disease_deid',
               'Asthma_Disease_Management_deid', 'Multiple_Sclerosis_deid',
               'Neuromuscular_Disease_deid', 'Coronary_Heart_Disease_deid',
               'Wound_Skin_deid', 'Sickle_Cell_Anemia_deid', 'HIV_Disease_deid',
               'High_Risk_Pregnancy_Disease_deid', 'Metabolic_Syndrome_deid',
               'Bipolar_Disorder_deid', 'Generalized_Anxiety_Disorder_deid',
               'Hypertension_deid', 'Chronic_Kidney_Disease_deid', 'Overweight_deid',
               'Cerebral_Cancer_deid', 'Breast_Cancer_Male_deid',
               'Obesity_Weight_deid', 'Chronic_Obstructive_Pulmonary_Disease_deid',
               'Heart_Failure_deid', 'Lung_Cancer_deid', 'Eating_Disorders_deid',
               'Gastrointestinal_deid', 'Colorectal_Cancer_deid', 'Pre_Diabetes_deid',
               'ADHD_deid', 'Obsessive_Compulsive_Neurosis_deid',
               'Borderline_Personality_Disorder_deid', 'Prostate_Cancer_deid',
               'Diabetes_Disease_deid', 'Schizophrenia_deid', 'Breast_Cancer_deid',
               'Substance_Abuse_deid', 'Low_Back_Pain_deid']

for col in looping_list:
    df2[col] = np.where(df2[col].isnull(), 0, 1)

# Replace "> 90" category with "90" so as to keep "age_deid" a continuous field
df2["age_deid"].replace({"> 90": "90"}, inplace = True)
df2["age_deid"] = pd.to_numeric(df2["age_deid"])
df3 = pd.get_dummies(df2)
df4 = df3.fillna(0)

# Scale continuous variables:
scaler    = StandardScaler()
cont_vars = ['clm_CCS_LVL_1', 'INPAT_OUTPAT_CD', 'claims_count', 'age_deid', 'Q1', 'Q2', 'Q3', 'Q4']
df4[cont_vars] = scaler.fit_transform(df4[cont_vars])

# Dedupe
df5 = df4.loc[:,~df4.columns.duplicated()]

# Sort by COVID period
cleaned = df5

# Sort df types by COVID flag, then remove the flag for modeling
pre_covid   = cleaned[cleaned['pre_COVID']== 1]
pre_covid   = pre_covid.drop(columns='pre_COVID')

in_covid  = cleaned[cleaned['pre_COVID']== 0]
in_covid  = in_covid.drop(columns='pre_COVID')

cleaned_all = cleaned.drop(columns='pre_COVID') 

# Export cleaned data that should be easier to use for modeling
# cleaned_all.to_csv(fp_all_data)
# pre_covid.to_csv(  fp_pre_covid)
# in_covid.to_csv(   fp_in_covid)

## Step 3: Prepare data for modeling

We prepared the data for modeling by including the following:
- Creating a function to generate the 'X' and 'y' arrays needed for scikit-learn with the ability to toggle the 'target' outcome and inclusion of Q1-Q4 risk scores.
- Generating a parameter grid for scikit-learn using the `LogisticRegression()` classifier to optimally tune hyperparameters 
- Splitting the data into 80/20 train/test sets using `random_state=4`

In [3]:
# Import data
data_all  = cleaned_all

def generate_xy(df, target, q4_only = False):
    """
    Desc: Use the original dataset to generate X and y for modeling part
    argv:
        df: Original dataframe
        target: The name of our y variable, like "reached" or "engaged"
        q4_only: If true, gives only the Q4 risk score
    return: X and y for modeling part
    """
    if q4_only == True:
        df1 = df.drop(['Q1', 'Q2', 'Q3'], axis =1)
    else:
        df1 = df
    
    df1[target] = np.where(df1[target] == True, 1, 0)
        
    X = df1.drop(target,axis = 1)
    y = df1[target]
    x_values = X.values
    y_values = y.values
    
    return X, y

# 'Engaged' target, including Q1-4
X, y = generate_xy(data_all , 'engaged', q4_only = False)

# Generate parameter grid for scikit-learn
pipe = Pipeline([('classifier' , RandomForestClassifier())])
param_grid = [
    
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__solver' : ['liblinear']
    }
    
]

# Set splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

## Step 4: Run the logistic regression model on cleaned data and obtain predicted values

In [4]:
# Create grid search object
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)

# Fit on data
best_clf = clf.fit(X_train, y_train)

# Create object of predicted values
y_pred = best_clf.predict(X_test)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.7min finished


## Step 5: Obtain prediction values and CV classification report

#### Parameters of the best estimator:

In [5]:
print(best_clf.best_params_)

{'classifier': LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}


#### 10-fold CV accuracy score of best estimator:

In [6]:
accuracy_score(y_test.values, y_pred)

0.7666265474552957

#### Confusion matrix of best estimator:

In [7]:
print(confusion_matrix(y_test,y_pred))

[[33282  4355]
 [ 9218 11305]]


#### Classification report of best estimator:

In [8]:
print(classification_report(y_test.values, y_pred, digits = 3))

             precision    recall  f1-score   support

          0      0.783     0.884     0.831     37637
          1      0.722     0.551     0.625     20523

avg / total      0.762     0.767     0.758     58160



## Step 6: Process output into prediction of users by 'most likely to engage'

#### Create table in descending order of 'most likely to engage'

In [9]:
# Create object of predicted probability of engagement by user
y_pred_prob_df = pd.DataFrame(best_clf.predict_proba(X_test))
y_pred_prob_df[1]
testDF = X_test

# Add column indicating probability of engagement by user
testDF['prob_engaging'] = y_pred_prob_df[1].values

# Reorder columns with probability as first column
cols = list(testDF.columns)
cols = [cols[-1]] + cols[:-1]
testDF = testDF[cols]

# Produce table with reordered columns in descending order of "Most Likely to Engage" by user
ordered_df = testDF.sort_values('prob_engaging', ascending=False)
ordered_df.head(5)

,prob_engaging,DHVD001,DHVD002,DHVD003,RLHC001,RROR001,DONC001,DERV002,DERV003,DJRX001,...,DERIVED_ETHNICITY_HISPANIC,DERIVED_ETHNICITY_NATIVE AMERICAN,DERIVED_ETHNICITY_PACIFIC ISLANDER,DERIVED_ETHNICITY_UNKNOWN,DERIVED_ETHNICITY_WHITE,DERIVED_FPL_CAT_133%,DERIVED_FPL_CAT_134-200%,DERIVED_FPL_CAT_201-400%,DERIVED_FPL_CAT_>400%,DERIVED_FPL_CAT_UNKNOWN
271307,0.977412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,1,0
176060,0.967954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
192030,0.963402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
274304,0.954823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
163718,0.951627,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


#### Extract coefficients from logistic regression model:

In [10]:
# Logistic Regression model coefficients in order of descending positive importance
coef_table = pd.DataFrame(list(X_train.columns)).copy()
coef_table.insert(len(coef_table.columns),"Coefs",best_clf.best_estimator_.named_steps["classifier"].coef_.transpose())
coef_table.columns = ['Variable', 'Coefficient']
coefficients = coef_table.reindex(coef_table.Coefficient.sort_values(ascending=False).index)
positive_coefficients = coefficients.head(104)
negative_coefficients = coefficients.tail(71).iloc[::-1]

#### Top 10 most influential *positive* coefficients in model:

In [38]:
positive_coefficients.head(10)

,Variable,Coefficient
67,condition_desc_Endocrine,6.173841
74,condition_desc_Neonatal,6.170428
73,condition_desc_Musculoskeletal,6.087576
65,condition_desc_Congenital,6.023554
76,condition_desc_Oncology,5.976982
63,condition_desc_Cardiovascular,5.888414
75,condition_desc_Neurologic,5.876970
62,condition_desc_Behavioral Health,5.826669
72,condition_desc_Maternity,5.702314
71,condition_desc_Injury and Poisoning,5.644771


#### Top 10 most influential *negative* coefficients in model:

In [37]:
negative_coefficients.head(10)

,Variable,Coefficient
84,mc_elgity_ind_No,-2.400541
87,state_NM,-1.324993
86,state_MT,-0.828320
4,RROR001,-0.752759
6,DERV002,-0.384332
1,DHVD002,-0.311454
14,DERV001,-0.310513
2,DHVD003,-0.290223
9,DHCT001,-0.288033
5,DONC001,-0.281527


## Step 7: Reorder variables from most to least influential *positive* coefficients in the model

In [12]:
# Produce table with reordered columns in descending order of "Most Likely to Engage" by user
ordered_df = testDF.sort_values('prob_engaging', ascending=False)

# Create list of most influential positive coefficients for output
positive_vars = ['prob_engaging']

for i, j in positive_coefficients.iterrows():
    positive_vars.append(j[0])
    
# Reorder variables to be in order of descending positive influence
reordered_df = ordered_df.reindex(columns = positive_vars)
reordered_df.reset_index(inplace=True)
reordered_df = reordered_df.rename(columns = {'index':'member_id'})

### Create function to select only top 5 factors (variables) per member

In [13]:
def export_top_factors_csv(input_df, csv_path):
    # input_df = reordered_df.head(50)
    """ 
    Function that takes in logit output summary, extracts top factors for each member and writes csv summary + returns pd summary
    
    Args:
        input_df: pd with 1 row per member, 1st column with prob of engaging, following columns with predictor values
        csv_path: desired path for csv summary
    
    """
    # Declaring new lists (used to create pd)
    member = []
    prob = []
    factor_1 = []
    factor_2 = []
    factor_3 = []
    factor_4 = []
    factor_5 = []

    for i, row in input_df.iterrows():
        member.append(row['member_id'])
        prob.append(row['prob_engaging'])
        list_variables = input_df.loc[i][2:][input_df.loc[i] > 0].index.tolist()
        number_factors = len(list_variables)

        if (len(list_variables)>=5):
            factor_1.append(list_variables[0])
            factor_2.append(list_variables[1])
            factor_3.append(list_variables[2])
            factor_4.append(list_variables[3])
            factor_5.append(list_variables[4])

        elif (len(list_variables)==4):
            factor_1.append(list_variables[0])
            factor_2.append(list_variables[1])
            factor_3.append(list_variables[2])
            factor_4.append(list_variables[3])
            factor_5.append("")

        elif (len(list_variables)==3):
            factor_1.append(list_variables[0])
            factor_2.append(list_variables[1])
            factor_3.append(list_variables[2])
            factor_4.append("")
            factor_5.append("")

        elif (len(list_variables)==2):
            factor_1.append(list_variables[0])
            factor_2.append(list_variables[1])
            factor_3.append("")
            factor_4.append("")
            factor_5.append("")

        elif (len(list_variables)==1):
            factor_1.append(list_variables[0])
            factor_2.append("")
            factor_3.append("")
            factor_4.append("")
            factor_5.append("")

        else:
            factor_1.append("")
            factor_2.append("")
            factor_3.append("")
            factor_4.append("")
            factor_5.append("")

    # Create pandas df
    dict = {'member_id':     member, 
            'prob_engaging': prob, 
            'factor_1':      factor_1, 
            'factor_2':      factor_2,
            'factor_3':      factor_3 , 
            'factor_4':      factor_4 ,
            'factor_5':      factor_5}
    summary = pd.DataFrame(dict)

    # Write summary to csv
    summary.to_csv(csv_path, index=False)   

    # Also return pd of summary
    return summary

### Output best member targets from **test data** with top 5 factors:

In [14]:
sample_output = export_top_factors_csv(reordered_df, fp_output)
sample_output.head(10)

,member_id,prob_engaging,factor_1,factor_2,factor_3,factor_4,factor_5
0,271307.0,0.977412,condition_desc_Communicable Disease,Q2,claims_count,Q1,Wound_Skin_deid
1,176060.0,0.967954,condition_desc_Congenital,Q2,claims_count,Q1,Obesity_Weight_deid
2,192030.0,0.963402,condition_desc_Oncology,Q2,claims_count,Q1,Obesity_Weight_deid
3,274304.0,0.954823,condition_desc_Cardiovascular,Q2,claims_count,Q1,
4,163718.0,0.951627,condition_desc_Congenital,Q2,claims_count,Q1,
5,144719.0,0.946455,condition_desc_Oncology,Q2,Q1,A8608_DwellingType_M,A8606_HmeownRenter_
6,23067.0,0.944958,condition_desc_Oncology,Q2,claims_count,Q1,Asthma_Disease_Management_deid
7,32526.0,0.943942,condition_desc_Oncology,Q2,claims_count,A9350_EconomicStabilityInd_17,Q1
8,46182.0,0.942112,condition_desc_Injury and Poisoning,Q2,claims_count,A9514_EducInputInd_3,Q1
9,231725.0,0.939382,condition_desc_Musculoskeletal,state_OK,Q2,claims_count,Q1
